In [109]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as  np
from scipy.spatial import distance
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
train_data_dir = './data'
w = 256
h = 256
batch_size = 1

n_epochs = 50

In [3]:
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='rgba',
    target_size=(w, h),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, 
    color_mode='rgba',
    target_size=(w, h),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation')



Found 684 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [94]:
def extract_features(img):
    R, G, B, A = cv2.split(img)
    alpha = A / 255
    R = (255 * (1 - alpha) + R * alpha).astype(np.uint8)
    G = (255 * (1 - alpha) + G * alpha).astype(np.uint8)
    B = (255 * (1 - alpha) + B * alpha).astype(np.uint8)
    img = cv2.merge((R, G, B))
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    orb = cv2.ORB_create(patchSize=21, edgeThreshold=21)
    return orb.detectAndCompute(gray, None)

In [112]:
def get_image_feature_db(generator):
    db = {}
    for i in range(generator.samples):
        x, y = next(generator)
        key = np.argmax(y[0])
        image = x[0]
        if key not in db.keys():
            db[key] = np.empty((1,32))
        
        keypoints, descriptors = extract_features(image)

        if descriptors is None:
            pass
        else:
            db[key] = np.vstack((db[key], descriptors))
    
    return db

train_db = get_image_feature_db(train_generator)
validation_db = get_image_feature_db(validation_generator)

In [114]:
def cluster(k, descriptor_list):
    kmeans = KMeans(n_clusters = k, n_jobs=10)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_
    return visual_words

descriptor_list = np.empty((1,32))
for key in train_db.keys():
    descriptor_list = np.vstack((descriptor_list, train_db[key]))
bow = cluster(20, descriptor_list) 

C:\Users\ancha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


In [120]:
len(train_db[0])

43094

In [121]:
def find_index(image, center):
    count = 0
    ind = 0
    for i in range(len(center)):
        if(i == 0):
           count = distance.euclidean(image, center[i]) 
           #count = L1_dist(image, center[i])
        else:
            dist = distance.euclidean(image, center[i]) 
            #dist = L1_dist(image, center[i])
            if(dist < count):
                ind = i
                count = dist
    return ind

def image_class(all_bovw, centers):
    dict_feature = {}
    for key,value in all_bovw.items():
        category = []
        for img in value:
            histogram = np.zeros(len(centers))
            for each_feature in img:
                ind = find_index(each_feature, centers)
                histogram[ind] += 1
            category.append(histogram)
        dict_feature[key] = category
    return dict_feature

bovw_train = image_class(d

Train
Test


NameError: name 'test_db' is not defined

In [ ]:
bovw_train = image_class(train_db, bow) 

In [127]:
bovw_test = image_class(validation_db, bow) 

In [126]:
bovw_train

,  0.,  0.,  0.,  0.,  0.,
          0.,  0., 10.,  0.,  0., 21.,  0.]),
  array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0., 17.,  0.,  0., 14.,  0.]),
  array([ 0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0., 10.,  0.,  0., 20.,  0.]),
  array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0., 13.,  0.,  0., 18.,  0.]),
  array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0., 17.,  0.,  0., 14.,  0.]),
  array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0., 14.,  0.,  0., 18.,  0.]),
  array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0., 16.,  0.,  0., 15.,  0.]),
  array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0., 16.,  0.,  0., 15.,  0.]),
  array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,

In [128]:
def knn(images, tests):
    num_test = 0
    correct_predict = 0
    class_based = {}
    
    for test_key, test_val in tests.items():
        class_based[test_key] = [0, 0] # [correct, all]
        for tst in test_val:
            predict_start = 0
            #print(test_key)
            minimum = 0
            key = "a" #predicted
            for train_key, train_val in images.items():
                for train in train_val:
                    if(predict_start == 0):
                        minimum = distance.euclidean(tst, train)
                        #minimum = L1_dist(tst,train)
                        key = train_key
                        predict_start += 1
                    else:
                        dist = distance.euclidean(tst, train)
                        #dist = L1_dist(tst,train)
                        if(dist < minimum):
                            minimum = dist
                            key = train_key
            
            if(test_key == key):
                correct_predict += 1
                class_based[test_key][0] += 1
            num_test += 1
            class_based[test_key][1] += 1
            #print(minimum)
    return [num_test, correct_predict, class_based]
    
# Call the knn function    
results_bowl = knn(bovw_train, bovw_test) 

KeyboardInterrupt: 

In [ ]:
def accuracy(results):
    avg_accuracy = (results[1] / results[0]) * 100
    print("Average accuracy: %" + str(avg_accuracy))
    print("\nClass based accuracies: \n")
    for key,value in results[2].items():
        acc = (value[0] / value[1]) * 100
        print(key + " : %" + str(acc))
        
# Calculates the accuracies and write the results to the console.       
accuracy(results_bowl) 